In [1]:
import boto3
import configparser
import json
import pandas as pd
import psycopg2

In [2]:
config=configparser.ConfigParser()
config.read_file(open('redshift-configuration.cfg'))

In [3]:
#IAM_USER
KEY=config.get('IAM_USER', 'KEY')
SECRET=config.get('IAM_USER','SECRET')
#IAM_ROLE
IAM_ROLE_NAME=config.get('IAM_ROLE', 'IAM_ROLE_NAME')
#CLUSTER-CONFIGURATION
CLUSTER_IDENTIFIER= config.get('CLUSTER-CONFIGURATION','CLUSTER_IDENTIFIER')
CLUSTER_TYPE      = config.get('CLUSTER-CONFIGURATION','CLUSTER_TYPE')
NODE_TYPE         = config.get('CLUSTER-CONFIGURATION','NODE_TYPE')
NUM_NODES         = config.get('CLUSTER-CONFIGURATION','NUM_NODES')
#DATABASE
DB_NAME     = config.get('DATABASE-CONFIGURATION','DB_NAME')
DB_USER     = config.get('DATABASE-CONFIGURATION','DB_USER')
DB_PASSWORD = config.get('DATABASE-CONFIGURATION','DB_PASSWORD')
DB_PORT     = config.get('DATABASE-CONFIGURATION','DB_PORT')

### Client for IAM, EC2, S3, and Redshift

In [4]:
ec2 = boto3.resource('ec2', 
                    region_name="us-west-2",
                    aws_access_key_id     = KEY,
                    aws_secret_access_key = SECRET
                    )

s3 = boto3.resource('s3',
                   region_name='us-west-2',
                   aws_access_key_id      = KEY,
                   aws_secret_access_key  = SECRET
                   )

iam = boto3.client('iam', 
                region_name='us-west-2',
                aws_access_key_id         = KEY,
                aws_secret_access_key     = SECRET
                )

redshift = boto3.client('redshift',
                      region_name='us-west-2',
                      aws_access_key_id    = KEY,
                      aws_secret_access_key= SECRET
                      )

In [5]:
from botocore.exceptions import ClientError

#1.1 Create the role
try:
    print("1.1 Creating a new IAM Role called 'IAM-FOR-data-modeling-on-Cloud'") 
    dwhRole = iam.create_role(
        Path='/',
        RoleName=IAM_ROLE_NAME,
        Description = "Allows Redshift clusters to call AWS services on your behalf.",
        AssumeRolePolicyDocument=json.dumps(
            {'Statement': [{'Action': 'sts:AssumeRole','Effect': 'Allow','Principal': {'Service': 'redshift.amazonaws.com'}}],
             'Version': '2012-10-17'})
    )    
except Exception as e:
    print(e)
    

print("1.2 Attaching Policy")

iam.attach_role_policy(RoleName=IAM_ROLE_NAME,PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess")['ResponseMetadata']['HTTPStatusCode']

print("1.3 Get the IAM role ARN")
roleArn = iam.get_role(RoleName=IAM_ROLE_NAME)['Role']['Arn']


print(roleArn)

1.1 Creating a new IAM Role called 'IAM-FOR-data-modeling-on-Cloud'
An error occurred (EntityAlreadyExists) when calling the CreateRole operation: Role with name iamRoleName-demo already exists.
1.2 Attaching Policy
1.3 Get the IAM role ARN
arn:aws:iam::185187464837:role/iamRoleName-demo


## STEP2: Redshift Cluster

### Create a Redshift Cluster

In [6]:
try:
    response=redshift.create_cluster(ClusterType       = CLUSTER_TYPE,
                                     NodeType          = NODE_TYPE,
                                     NumberOfNodes     = int(NUM_NODES),
                                     DBName            = DB_NAME,
                                     ClusterIdentifier = CLUSTER_IDENTIFIER,
                                     MasterUsername    = DB_USER,
                                     MasterUserPassword= DB_PASSWORD,
                                     #Roles ( for s3 access)
                                     IamRoles= [roleArn]
                                        )
except Exception as e : 
    print(e)

In [7]:
def description (props):
    pd.set_option('display.max_colwidth',None)
    keyToShow= ["ClusterIdentifier", "NodeType", "ClusterStatus", "MasterUsername", "DBName", "Endpoint", "NumberOfNodes", 'VpcId']
    x = [(k,v) for k,v in props.items() if k in keyToShow]
    return pd.DataFrame(data=x,columns=['Key','Value'])


In [10]:
myClusterProps = redshift.describe_clusters(ClusterIdentifier=CLUSTER_IDENTIFIER)['Clusters'][0]
description( myClusterProps)

,Key,Value
0,ClusterIdentifier,data-modeling-cloud
1,NodeType,dc2.large
2,ClusterStatus,available
3,MasterUsername,admin
4,DBName,database
5,Endpoint,"{'Address': 'data-modeling-cloud.cahjwy6ny05j.us-west-2.redshift.amazonaws.com', 'Port': 5439}"
6,VpcId,vpc-61eca419
7,NumberOfNodes,4


In [11]:
ENDPOINT = myClusterProps['Endpoint']['Address']
ROLE_ARN = myClusterProps['IamRoles'][0]['IamRoleArn']

In [13]:
# write the endpoint to redshift-configuration.cfg
with open('redshift-configuration.cfg', 'r') as file:
    data = file.readlines()
data[14]="HOST={}\n".format(ENDPOINT)

with open('redshift-configuration.cfg', 'w') as f:
    f.writelines( data )


In [14]:
# this one is functional ( les parametre dans cfg sont dans l'order pour CLUSTER ) 
conn = psycopg2.connect("host={} dbname={} user={} password={} port={}".format(*config['DATABASE-CONFIGURATION'].values()))
cur=conn.cursor()

## Clean up the AWS resources

In [15]:
redshift.delete_cluster(ClusterIdentifier= CLUSTER_IDENTIFIER, SkipFinalClusterSnapshot= True)

{'Cluster': {'ClusterIdentifier': 'data-modeling-cloud',
  'NodeType': 'dc2.large',
  'ClusterStatus': 'deleting',
  'ClusterAvailabilityStatus': 'Modifying',
  'MasterUsername': 'admin',
  'DBName': 'database',
  'Endpoint': {'Address': 'data-modeling-cloud.cahjwy6ny05j.us-west-2.redshift.amazonaws.com',
   'Port': 5439},
  'ClusterCreateTime': datetime.datetime(2021, 1, 25, 15, 40, 38, 375000, tzinfo=tzutc()),
  'AutomatedSnapshotRetentionPeriod': 1,
  'ManualSnapshotRetentionPeriod': -1,
  'ClusterSecurityGroups': [],
  'VpcSecurityGroups': [{'VpcSecurityGroupId': 'sg-1d670a33',
    'Status': 'active'}],
  'ClusterParameterGroups': [{'ParameterGroupName': 'default.redshift-1.0',
    'ParameterApplyStatus': 'in-sync'}],
  'ClusterSubnetGroupName': 'default',
  'VpcId': 'vpc-61eca419',
  'AvailabilityZone': 'us-west-2c',
  'PreferredMaintenanceWindow': 'fri:08:00-fri:08:30',
  'PendingModifiedValues': {},
  'ClusterVersion': '1.0',
  'AllowVersionUpgrade': True,
  'NumberOfNodes': 4,


In [16]:
myClusterProps=redshift.describe_clusters(ClusterIdentifier= CLUSTER_IDENTIFIER)['Clusters'][0]
description( myClusterProps)

,Key,Value
0,ClusterIdentifier,data-modeling-cloud
1,NodeType,dc2.large
2,ClusterStatus,deleting
3,MasterUsername,admin
4,DBName,database
5,Endpoint,"{'Address': 'data-modeling-cloud.cahjwy6ny05j.us-west-2.redshift.amazonaws.com', 'Port': 5439}"
6,VpcId,vpc-61eca419
7,NumberOfNodes,4
